In [3]:
# Importing libraries
import pandas as pd
def recommendation():
    
    # Read csv file into a pandas dataframe
    df = pd.read_csv("books.csv")
    df = df[df.language_code =='eng']
    df['Title'] = df['title'].str.split('(').str[0]
    print("Enter rating w/wo <rating> or random for book suggestion")
    search= input("enter to search ")
    if search=="random":
        df_new= df.sample(replace=True)[['Title', 'authors',"rating"]]
        print("Title: ", df_new[['Title']].to_string(index=False, header=False))
        print("Author: ", df_new[['authors']].to_string(index=False, header=False))
        print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
        return "Enjoy!"
    else:
        if "rating" in search:
            search=search.split()
            rate=float(search[1])
            
            if rate < 0 or rate > 5:
                print("Error: Enter rating between 0 and 5")
                return "Please Correct"
            else:
                df_rate = df[df['rating'].astype(float) >= rate]
                df_new= df_rate.sample(replace=True)[['Title', 'authors',"rating"]]
                print("Title: ", df_new[['Title']].to_string(index=False, header=False))
                print("Author: ", df_new[['authors']].to_string(index=False, header=False))
                print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
                return "Enjoy!"
print(recommendation())

Enter rating w/wo <rating> or random for book suggestion
enter to search random
Title:   How to Talk to Anyone: 92 Little Tricks for Big Success in Relationships
Author:   Leil Lowndes
Rating:   3.78
Enjoy!


In [23]:
def range_recommend():
    '''function to recommend a book within an entered range'''
    # Read csv file into a pandas dataframe
    df = pd.read_csv("books.csv")
    df = df[df.language_code =='eng']
    print("Enter rating range between 1 and 5: ")
    
    llimit= float(input("enter lower limit "))
    if llimit<1 or llimit>5:
        llimit=1
    ulimit= float(input("enter upper limit"))
    if ulimit>5 or ulimit<llimit:
        ulimit=5
    df_rate = df[(df['rating'].astype(float) >= llimit) & (df['rating'].astype(float) <= ulimit) ]
    df_new= df_rate.sample(replace=True)[['title', 'authors',"rating"]]
    print("Title: ", df_new[['title']].to_string(index=False, header=False))
    print("Author: ", df_new[['authors']].to_string(index=False, header=False))
    print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
    return "Enjoy!"
range_recommend()

Enter rating range between 1 and 5: 
enter lower limit 2
enter upper limit3
Title:  Healthy Cooking for IBS: 100 Delicious Recipes to Keep You Symptom-Free
Author:  Sophie Braimbridge/Erica Jankovich
Rating:  2.62


'Enjoy!'

In [18]:
import pandas as pd
def author_recommend():
   
    df = pd.read_csv("books.csv")
    author=input("Enter the name of the author to search: ");
    df = df[df.authors ==author.title()]
    
    if(df.empty):
        print("There is no book with the specefied author")
    else:
        print(df[['title','rating']])
author_recommend()

Enter the name of the author to search: bill bryson
                                                   title rating
12                  A Short History of Nearly Everything   4.21
13                           Bill Bryson's African Diary   3.44
14     Bryson's Dictionary of Troublesome Words: A Wr...   3.87
15                                In a Sunburned Country   4.07
16     I'm a Stranger Here Myself: Notes on Returning...    3.9
17     The Lost Continent: Travels in Small Town America   3.83
18             Neither Here nor There: Travels in Europe   3.86
19                             Notes from a Small Island   3.91
20     The Mother Tongue: English and How It Got That...   3.93
1161                A Short History of Nearly Everything   4.21
1458   A Short History of Nearly Everything (Illustra...   4.21
2661   A Walk in the Woods: Rediscovering America on ...   4.06
2662   A Walk in the Woods: Rediscovering America on ...   4.06
2834   The Lost Continent:  Travels in Small-town Am